<a href="https://colab.research.google.com/github/Recepurkun/GMKA_Recep_Urkun/blob/master/Odevi4_Python_Veri_Analizi_Egitimi__Recep_%C3%9Crk%C3%BCn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Güney Marmara ve TRAI
# <center> Python Veri Analizi Eğitimi
# <center> Ödev 4

In [ ]:
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from plotly.tools import make_subplots

import warnings
warnings.filterwarnings("ignore")

### Veriyi özetleme

1.  3. Ders sonunda hazırladığınız 2021 ve 2022 yıllarına ait BIST30 ve EUR/USD kur değerlerini içeren bileşke tablodan sadece ayın son işgününe ait değerleri içeren bir pivot tablo üretiniz.

In [ ]:
#2022 BIST30 ve EUR/USD kur degerlerini indiriyoruz
indirme_listesi = ["XU030.IS","EURUSD=X"]
Veri_21_22 = pd.DataFrame()

for i in indirme_listesi:
  data = yf.download(i, start = "2021-01-01", end= "2023-01-01")
  data = data["Adj Close"].rename(i)
  Veri_21_22 = pd.concat([Veri_21_22, data], axis=1) #verilen 2 veriyi sutun olarak birlestirme islemi

Veri_21_22.columns = ["Kapanis (TRY)","EUR/USD"]#satir isimlendirmelerini degistiriyoruz
Veri_21_22 = Veri_21_22[Veri_21_22["Kapanis (TRY)"]>0]#borsanin kapali oldugu zaman dilimini tablomuzdan siliyoruz.
Veri_21_22

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Kapanis (TRY),EUR/USD
2021-01-04,1652.900024,1.225070
2021-01-05,1640.400024,1.225160
2021-01-06,1657.699951,1.230027
2021-01-07,1678.199951,1.234111
2021-01-08,1694.400024,1.227144
...,...,...
2022-12-23,5885.200195,1.060333
2022-12-27,5876.700195,1.064022
2022-12-28,5789.399902,1.063717
2022-12-29,5915.200195,1.062925


In [ ]:
#'Index' object has no attribute 'strftime'; hatasi verdigi icin index'i stringden datetime ogesine cevirdik
Veri_21_22.index = pd.to_datetime(Veri_21_22.index) # convert index to datetime object
Veri_21_22['Ay_Yil'] = (Veri_21_22.index).strftime('%Y-%m')
Veri_21_22

,Kapanis (TRY),EUR/USD,Ay_Yil
2021-01-04,1652.900024,1.225070,2021-01
2021-01-05,1640.400024,1.225160,2021-01
2021-01-06,1657.699951,1.230027,2021-01
2021-01-07,1678.199951,1.234111,2021-01
2021-01-08,1694.400024,1.227144,2021-01
...,...,...,...
2022-12-23,5885.200195,1.060333,2022-12
2022-12-27,5876.700195,1.064022,2022-12
2022-12-28,5789.399902,1.063717,2022-12
2022-12-29,5915.200195,1.062925,2022-12


In [ ]:
##aylik degisim hesaplamalari
Veri_21_22["degisim"] = Veri_21_22["Kapanis (TRY)"].diff(1)
Veri_21_22["d_yon"] = Veri_21_22["degisim"].map(lambda x: 'dusen gun' if x < 0 else 'yukselen gun') 
Veri_21_22["degisim%"] = (Veri_21_22["degisim"]/Veri_21_22["Kapanis (TRY)"].shift(1))#*100 grafikte daha alisildik degerleri gorebilmemiz adina *100 kismini siliyoruz
Veri_21_22

,Kapanis (TRY),EUR/USD,Ay_Yil,degisim,d_yon,degisim%
2021-01-04,1652.900024,1.225070,2021-01,NaN,yukselen gun,NaN
2021-01-05,1640.400024,1.225160,2021-01,-12.500000,dusen gun,-0.007562
2021-01-06,1657.699951,1.230027,2021-01,17.299927,yukselen gun,0.010546
2021-01-07,1678.199951,1.234111,2021-01,20.500000,yukselen gun,0.012367
2021-01-08,1694.400024,1.227144,2021-01,16.200073,yukselen gun,0.009653
...,...,...,...,...,...,...
2022-12-23,5885.200195,1.060333,2022-12,2.000000,yukselen gun,0.000340
2022-12-27,5876.700195,1.064022,2022-12,-8.500000,dusen gun,-0.001444
2022-12-28,5789.399902,1.063717,2022-12,-87.300293,dusen gun,-0.014855
2022-12-29,5915.200195,1.062925,2022-12,125.800293,yukselen gun,0.021729


In [ ]:
#pivot tablo olusturma
Veri_aysonu = Veri_21_22.pivot_table(values=["Kapanis (TRY)","EUR/USD"],index="Ay_Yil",aggfunc="last")#kapanis ve usd/try degerini al, 
#index yani tarihe["ay_yil"] gore yap bu islemi diyoruz. Agg func=> Ay sonu degerlerini istedigimiz icin de last diyoruz.
Veri_aysonu

,EUR/USD,Kapanis (TRY)
Ay_Yil,,
2021-01,1.212004,1572.400024
2021-02,1.216042,1556.000000
2021-03,1.172360,1430.699951
2021-04,1.212709,1456.900024
2021-05,1.219319,1506.900024
2021-06,1.190193,1462.900024
2021-07,1.189300,1500.800049
2021-08,1.179690,1588.300049
2021-09,1.160160,1505.199951


2. Ay kırılımlı olarak ürettiğiniz tabloyu grafik olarak inceleyiniz.


In [ ]:
##aylik degisim hesaplamalari
Veri_aysonu["degisim"] = Veri_aysonu["Kapanis (TRY)"].diff(1)
Veri_aysonu["d_yon"] = Veri_aysonu["degisim"].map(lambda x: 'dusen gun' if x < 0 else 'yukselen gun') 
Veri_aysonu["degisim%"] = (Veri_aysonu["degisim"]/Veri_aysonu["Kapanis (TRY)"].shift(1))#*100 grafikte daha alisildik degerleri gorebilmemiz adina *100 kismini siliyoruz
Veri_aysonu

,EUR/USD,Kapanis (TRY),degisim,d_yon,degisim%
Ay_Yil,,,,,
2021-01,1.212004,1572.400024,NaN,yukselen gun,NaN
2021-02,1.216042,1556.000000,-16.400024,dusen gun,-0.010430
2021-03,1.172360,1430.699951,-125.300049,dusen gun,-0.080527
2021-04,1.212709,1456.900024,26.200073,yukselen gun,0.018313
2021-05,1.219319,1506.900024,50.000000,yukselen gun,0.034319
2021-06,1.190193,1462.900024,-44.000000,dusen gun,-0.029199
2021-07,1.189300,1500.800049,37.900024,yukselen gun,0.025907
2021-08,1.179690,1588.300049,87.500000,yukselen gun,0.058302
2021-09,1.160160,1505.199951,-83.100098,dusen gun,-0.052320


In [ ]:
px.bar(Veri_aysonu, y="degisim%",
       color = "d_yon", color_discrete_map={"dusen gun" : "red", "yukselen gun" : "green"},
       text_auto=".0%")

# ***2021 Mart ve Eylul, 2022 Haziran aylarinda %5'den fazla dususler yasamasina ragmen geri kalan aylarda ozellikle 2022 Agustos, Ekim ve Kasim aylarinda ise %25e varan yukselisler gostermistir.***

3. Ders sonunda hazırladığınız 2021 ve 2022 yıllarına ait BIST30 ve EUR/USD kur değerlerini içeren bileşke tablodan sadece ayın son işgününe ait değerleri içeren bir tabloyu bir de resample yöntemi ile üretiniz.

In [ ]:
#Veri_21_22 icerisinde NaN olan herhangi bir deger yokken burada neden NaN degerler geliyor???
Veri_aysonu2 = Veri_21_22.resample("BM",convention="end").asfreq()
Veri_aysonu2

,Kapanis (TRY),EUR/USD,Ay_Yil,degisim,d_yon,degisim%
2021-01-29,1572.400024,1.212004,2021-01,-29.500000,dusen gun,-0.018416
2021-02-26,1556.000000,1.216042,2021-02,-18.099976,dusen gun,-0.011499
2021-03-31,1430.699951,1.172360,2021-03,-1.200073,dusen gun,-0.000838
2021-04-30,1456.900024,1.212709,2021-04,-8.699951,dusen gun,-0.005936
2021-05-31,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-30,1462.900024,1.190193,2021-06,-20.099976,dusen gun,-0.013554
2021-07-30,1500.800049,1.189300,2021-07,0.900024,yukselen gun,0.000600
2021-08-31,1588.300049,1.179690,2021-08,14.300049,yukselen gun,0.009085
2021-09-30,1505.199951,1.160160,2021-09,16.299927,yukselen gun,0.010948
2021-10-29,NaN,NaN,NaN,NaN,NaN,NaN


4. Resample yöntemi ile ürettiğiniz tabloyu grafik olarak inceleyiniz. 2. adımda elde ettiğiniz grafik ile karşılaştırarak yorumlayınız.

In [ ]:
px.bar(Veri_aysonu2, y="degisim%",
       color = "d_yon", color_discrete_map={"dusen gun" : "red", "yukselen gun" : "green"},
       text_auto=".0%")